In [1]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

Instantiaite SparkSession with hive support 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

In [3]:
customerDF = spark.read.load("/home/hduser/Downloads/sharedfolder/customers.tsv",format="csv", sep="\t", inferSchema="true")
salestxnDF = spark.read.load("/home/hduser/Downloads/sharedfolder/salestxns.tsv",format="csv", sep="\t", inferSchema="true")

In [4]:
customerDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)



In [5]:
salestxnDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)



In [6]:
new_names_cDF = ['customer_id','customer_name','customer_city','customer_state','customer_zipcode']

In [7]:
new_names_sDF= ['salestxn_ID','category_id','category_name','product_id','product_name','product_price',\
               'product_quantity','customer_id']

In [9]:
cDF2=customerDF.toDF(*new_names_cDF)

In [10]:
sDF2=salestxnDF.toDF(*new_names_sDF)

In [11]:
cDF2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [12]:
cDF2.show(5)

+-----------+--------------+-------------+--------------+----------------+
|customer_id| customer_name|customer_city|customer_state|customer_zipcode|
+-----------+--------------+-------------+--------------+----------------+
|      11039|   Mary Torres|       Caguas|            PR|             725|
|       5623|    Jose Haley|     Columbus|            OH|           43207|
|       5829|    Mary Smith|      Houston|            TX|           77015|
|       6336|Richard Maddox|       Caguas|            PR|             725|
|       1708|Margaret Booth|    Arlington|            TX|           76010|
+-----------+--------------+-------------+--------------+----------------+
only showing top 5 rows



### Broadcast JOIN

In [14]:
JDF1 = cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [15]:
JDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain()

== Physical Plan ==
*BroadcastHashJoin [customer_id#52], [customer_id#70], Inner, BuildLeft
:- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
:  +- *Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
:     +- *Filter isnotnull(_c0#12)
:        +- *FileScan csv [_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/Downloads/sharedfolder/customers.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(_c0)], ReadSchema: struct<_c0:int,_c1:string,_c2:string,_c3:string,_c4:int>
+- *Project [_c0#35 AS salestxn_ID#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
   +- *Filter isnotnull(_c7#42)
      +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_

In [16]:
JDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#52 = customer_id#70)
:- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
:  +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv
+- Project [_c0#35 AS salestxn_ID#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
   +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv

== Analyzed Logical Plan ==
customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int, salestxn_ID: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int
Join Inner, (customer_id#52 = customer_id#70)
:- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _

In [17]:
JDF2 = sDF2.join(cDF2, sDF2.customer_id==cDF2.customer_id)

In [18]:
type(JDF2)

pyspark.sql.dataframe.DataFrame

In [19]:
JDF2.explain()

== Physical Plan ==
*BroadcastHashJoin [customer_id#70], [customer_id#52], Inner, BuildRight
:- *Project [_c0#35 AS salestxn_ID#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- *Filter isnotnull(_c7#42)
:     +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/Downloads/sharedfolder/salestxns.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(_c7)], ReadSchema: struct<_c0:int,_c1:int,_c2:string,_c3:int,_c4:string,_c5:double,_c6:int,_c7:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
   +- *Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- *Filter isnotnull(_c0#12)
         +- *File

In [20]:
JDF2.explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_ID#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
   +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_ID: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_ID#63, _c1#36 AS category_id#64, _c2

In [21]:
JDF3 = sDF2.join(cDF2.hint("broadcast"),sDF2.customer_id==cDF2.customer_id).explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_ID#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- ResolvedHint isBroadcastable=true
   +- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_ID: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS sa

In [22]:
JDF3 = sDF2.join(cDF2.hint("broadcast"),sDF2.customer_id==cDF2.customer_id)

In [23]:
JDF3.printSchema()

root
 |-- salestxn_ID: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [24]:
JDF3.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|salestxn_ID|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|customer_id|customer_name|customer_city|customer_state|customer_zipcode|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|         58|         45|         Fishing|      1004|Field & Stream Sp...|       399.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         59|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         78|          9|Cardio Equipment|       1

In [25]:
pyspark.sql.functions.broadcast(cDF2)

DataFrame[customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int]

In [26]:
type(pyspark.sql.functions.broadcast(cDF2))

pyspark.sql.dataframe.DataFrame

In [27]:
JDF4 = sDF2.join(pyspark.sql.functions.broadcast(cDF2),sDF2.customer_id==cDF2.customer_id)
type(JDF4)

pyspark.sql.dataframe.DataFrame

In [28]:
JDF4.explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_ID#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- ResolvedHint isBroadcastable=true
   +- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_ID: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS sa

In [29]:
JDF4.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|salestxn_ID|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|customer_id|customer_name|customer_city|customer_state|customer_zipcode|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|         58|         45|         Fishing|      1004|Field & Stream Sp...|       399.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         59|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         78|          9|Cardio Equipment|       1